# TopK-Airlines: Experimental Results Analysis

This notebook analyzes the performance of different data structures (Heap, Balanced BST, Skip List) for maintaining Top-K airlines by average rating.

## Experiment Overview
- **Dataset**: Skytrax airline reviews
- **Operations**: Insert, Search, Delete
- **Metrics**: Runtime, Memory usage, Top-K accuracy
- **Data Structures**: Min/Max Heap, Balanced BST, Skip List

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.append('..')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Setup complete!")

## Load Experimental Results

In [ ]:
# Load results from experiments
results_path = Path('../results/results.csv')

if results_path.exists():
    results_df = pd.read_csv(results_path)
    print(f"Loaded {len(results_df)} experimental results")
    display(results_df.head(10))
else:
    print("⚠️ No results found. Please run experiments first using: python src/experiment.py")
    results_df = pd.DataFrame()

## Data Summary Statistics

In [ ]:
if not results_df.empty:
    print("\n=== Summary Statistics ===")
    print(f"\nData structures tested: {results_df['structure'].unique()}")
    print(f"\nOperations tested: {results_df['operation'].unique() if 'operation' in results_df.columns else 'N/A'}")
    print(f"\nData size range: {results_df['n'].min()} - {results_df['n'].max()}")
    
    # Group statistics
    summary = results_df.groupby('structure').agg({
        'avg_time': ['mean', 'std', 'min', 'max']
    }).round(6)
    display(summary)

## Visualization 1: Runtime Comparison Across Data Structures

In [ ]:
if not results_df.empty:
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot each data structure
    structures = results_df['structure'].unique()
    colors = plt.cm.Set2(np.linspace(0, 1, len(structures)))
    
    for idx, structure in enumerate(structures):
        data = results_df[results_df['structure'] == structure].sort_values('n')
        ax.plot(data['n'], data['avg_time'], 
                marker='o', linewidth=2, markersize=8,
                label=structure, color=colors[idx], alpha=0.8)
    
    ax.set_xlabel('Number of Operations (n)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Time (seconds)', fontsize=12, fontweight='bold')
    ax.set_title('Runtime Comparison: Top-K Data Structures', fontsize=14, fontweight='bold', pad=20)
    ax.legend(fontsize=11, loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/runtime_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Plot saved to results/runtime_comparison.png")

## Visualization 2: Runtime Comparison by Operation Type

In [ ]:
if not results_df.empty and 'operation' in results_df.columns:
    operations = results_df['operation'].unique()
    n_ops = len(operations)
    
    fig, axes = plt.subplots(1, n_ops, figsize=(6*n_ops, 5))
    if n_ops == 1:
        axes = [axes]
    
    for idx, operation in enumerate(operations):
        ax = axes[idx]
        op_data = results_df[results_df['operation'] == operation]
        
        for structure in op_data['structure'].unique():
            data = op_data[op_data['structure'] == structure].sort_values('n')
            ax.plot(data['n'], data['avg_time'], 
                   marker='o', linewidth=2, label=structure, alpha=0.8)
        
        ax.set_xlabel('Number of Operations (n)', fontsize=11)
        ax.set_ylabel('Average Time (seconds)', fontsize=11)
        ax.set_title(f'{operation.capitalize()} Operation', fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/runtime_by_operation.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Plot saved to results/runtime_by_operation.png")

## Visualization 3: Performance Heatmap

In [ ]:
if not results_df.empty:
    # Create pivot table for heatmap
    pivot_data = results_df.pivot_table(
        values='avg_time', 
        index='structure', 
        columns='n', 
        aggfunc='mean'
    )
    
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.imshow(pivot_data.values, cmap='YlOrRd', aspect='auto')
    
    # Set ticks and labels
    ax.set_xticks(np.arange(len(pivot_data.columns)))
    ax.set_yticks(np.arange(len(pivot_data.index)))
    ax.set_xticklabels(pivot_data.columns)
    ax.set_yticklabels(pivot_data.index)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Average Time (seconds)', fontsize=11)
    
    # Add values to cells
    for i in range(len(pivot_data.index)):
        for j in range(len(pivot_data.columns)):
            text = ax.text(j, i, f'{pivot_data.values[i, j]:.4f}',
                         ha="center", va="center", color="black", fontsize=9)
    
    ax.set_xlabel('Number of Operations (n)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Data Structure', fontsize=12, fontweight='bold')
    ax.set_title('Performance Heatmap: Average Runtime', fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig('../results/performance_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Plot saved to results/performance_heatmap.png")

## Visualization 4: Scalability Analysis (Log Scale)

In [ ]:
if not results_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Linear scale
    for structure in results_df['structure'].unique():
        data = results_df[results_df['structure'] == structure].sort_values('n')
        ax1.plot(data['n'], data['avg_time'], marker='o', label=structure, linewidth=2)
    
    ax1.set_xlabel('Number of Operations (n)', fontsize=11)
    ax1.set_ylabel('Average Time (seconds)', fontsize=11)
    ax1.set_title('Linear Scale', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Log-log scale
    for structure in results_df['structure'].unique():
        data = results_df[results_df['structure'] == structure].sort_values('n')
        ax2.loglog(data['n'], data['avg_time'], marker='o', label=structure, linewidth=2)
    
    ax2.set_xlabel('Number of Operations (n)', fontsize=11)
    ax2.set_ylabel('Average Time (seconds)', fontsize=11)
    ax2.set_title('Log-Log Scale', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3, which='both')
    
    plt.suptitle('Scalability Analysis', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('../results/scalability_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Plot saved to results/scalability_analysis.png")

## Analysis & Conclusions

### Key Findings

#### 1. **Heap Performance**
- Best for: 
- Time Complexity: O(log n) for insert/delete, O(1) for peek
- Observations:

#### 2. **Balanced BST Performance**
- Best for:
- Time Complexity: O(log n) for all operations
- Observations:

#### 3. **Skip List Performance**
- Best for:
- Time Complexity: O(log n) average case
- Observations:

### Recommendations
- **Use Heap when**: You primarily need to find min/max and occasional insertions
- **Use BST when**: You need balanced performance across all operations
- **Use Skip List when**: You need probabilistic guarantees and easier implementation

### Future Work
- [ ] Test with larger datasets (1M+ reviews)
- [ ] Implement memory usage tracking
- [ ] Add concurrent access benchmarks
- [ ] Test with different K values (Top-5, Top-20, Top-50)
- [ ] Analyze cache performance

## Export Summary Report

In [ ]:
if not results_df.empty:
    # Generate summary report
    report = []
    report.append("=" * 60)
    report.append("TopK-Airlines Experiment Summary Report")
    report.append("=" * 60)
    report.append(f"\nTotal experiments run: {len(results_df)}")
    report.append(f"Data structures tested: {', '.join(results_df['structure'].unique())}")
    report.append(f"\nPerformance Summary (average runtime in seconds):")
    report.append("-" * 60)
    
    for structure in results_df['structure'].unique():
        avg_time = results_df[results_df['structure'] == structure]['avg_time'].mean()
        report.append(f"{structure:20s}: {avg_time:.6f}")
    
    report.append("\n" + "=" * 60)
    
    report_text = "\n".join(report)
    print(report_text)
    
    # Save report
    with open('../results/summary_report.txt', 'w') as f:
        f.write(report_text)
    
    print("\n✅ Summary report saved to results/summary_report.txt")